# Udacity Course - Intro to Machine Learning

## Final Project Overview:

In this project, you will play detective, and put your machine learning skills to use by building an algorithm to identify Enron Employees who may have committed fraud based on the public Enron financial and email dataset.

#### Project Introduction
In 2000, Enron was one of the largest companies in the United States. By 2002, it had collapsed into bankruptcy due to widespread corporate fraud. In the resulting Federal investigation, there was a significant amount of typically confidential information entered into public record, including tens of thousands of emails and detailed financial data for top executives. In this project, you will play detective, and put your new skills to use by building a person of interest identifier based on financial and email data made public as a result of the Enron scandal. To assist you in your detective work, we've combined this data with a hand-generated list of persons of interest in the fraud case, which means individuals who were indicted, reached a settlement, or plea deal with the government, or testified in exchange for prosecution immunity.

In [45]:

import sys
import pickle
import numpy as np
import pandas as pd
import sklearn
#from ggplot import *
import matplotlib as plt
%matplotlib inline

sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data


### Task 1: Features Selection

The features in the data fall into three major types, namely financial features, email features and POI labels.

financial features: ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees'] (all units are in US dollars)

email features: ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi'] (units are generally number of emails messages; notable exception is ‘email_address’, which is a text string)

POI label: [‘poi’] (boolean, represented as integer)

I'm going to try to use all features, filter them and choose the best.

In [10]:

### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list = ['poi', 'salary', 'to_messages', 'deferral_payments', 'total_payments', 
                 'loan_advances', 'bonus', 'restricted_stock_deferred', 
                 'deferred_income', 'total_stock_value', 'expenses', 'from_poi_to_this_person', 
                 'exercised_stock_options', 'from_messages', 'other', 'from_this_person_to_poi', 
                 'long_term_incentive', 'shared_receipt_with_poi', 'restricted_stock', 'director_fees'] 

#financial_features_list = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred', 'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other', 'long_term_incentive', 'restricted_stock', 'director_fees']
#email_features_list = ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']


### Load the dictionary containing the dataset
with open("final_project_dataset_modified.pkl", "r") as data_file:
    enron_data = pickle.load(data_file)
    
enron_data['METTS MARK'].keys()

f = open('poi_names.txt', 'r')



In [13]:
df = pd.DataFrame.from_records(list(enron_data.values()))
persons = pd.Series(list(enron_data.keys()))

print persons.head()
print df.head()

0          METTS MARK
1       BAXTER JOHN C
2      ELLIOTT STEVEN
3    CORDES WILLIAM R
4      HANNON KEVIN P
dtype: object
     bonus deferral_payments deferred_income director_fees  \
0   600000               NaN             NaN           NaN   
1  1200000           1295738        -1386055           NaN   
2   350000               NaN         -400729           NaN   
3      NaN               NaN             NaN           NaN   
4  1500000               NaN        -3117011           NaN   

              email_address exercised_stock_options expenses from_messages  \
0      mark.metts@enron.com                     NaN    94299            29   
1                       NaN                 6680544    11200           NaN   
2  steven.elliott@enron.com                 4890344    78552           NaN   
3     bill.cordes@enron.com                  651850      NaN            12   
4    kevin.hannon@enron.com                 5538001    34039            32   

  from_poi_to_this_person from_thi

### Task 2: Remove outliers

In [15]:
# convert to numpy.nan
df.replace(to_replace='NaN', value=np.nan, inplace=True)

# count number of nan for columns
print df.isnull().sum()

# dataframe dimension
df.shape

bonus                         64
deferral_payments            106
deferred_income               96
director_fees                127
email_address                 34
exercised_stock_options       44
expenses                      51
from_messages                 59
from_poi_to_this_person       59
from_this_person_to_poi       59
loan_advances                141
long_term_incentive           80
other                         53
poi                            0
restricted_stock              36
restricted_stock_deferred    126
salary                        51
shared_receipt_with_poi       59
to_messages                   59
total_payments                21
total_stock_value             20
dtype: int64


(143, 21)

In [24]:
# remove column from df if null counter > 65
for column, series in df.iteritems():
    if series.isnull().sum() > 65:
        df.drop(column, axis=1, inplace=True)

# remove email address column
if 'email_address' in list(df.columns.values):
    df.drop('email_address', axis=1, inplace=True)



In [ ]:
df.info()

In [27]:
# Impute the missing values
from sklearn.preprocessing import Imputer
#imp = Imputer(missing_values='NaN', strategy='median', axis=0)
#imp.fit(df)
#df_imp = pd.DataFrame(imp.transform(df.copy(deep=True)))

df_imp = df.replace(to_replace=np.nan, value=0)
df_imp = df.fillna(0).copy(deep=True)
df_imp.columns = list(df.columns.values)

print df_imp.isnull().sum()
print df_imp.head()

df_imp.describe()


bonus                      0
exercised_stock_options    0
expenses                   0
from_messages              0
from_poi_to_this_person    0
from_this_person_to_poi    0
other                      0
poi                        0
restricted_stock           0
salary                     0
shared_receipt_with_poi    0
to_messages                0
total_payments             0
total_stock_value          0
dtype: int64
       bonus  exercised_stock_options  expenses  from_messages  \
0   600000.0                      0.0   94299.0           29.0   
1  1200000.0                6680544.0   11200.0            0.0   
2   350000.0                4890344.0   78552.0            0.0   
3        0.0                 651850.0       0.0           12.0   
4  1500000.0                5538001.0   34039.0           32.0   

   from_poi_to_this_person  from_this_person_to_poi      other    poi  \
0                     38.0                      1.0     1740.0  False   
1                      0.0            

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,1.430000e+02,1.430000e+02,143.000000,143.000000,143.000000,143.000000,1.430000e+02,1.430000e+02,1.430000e+02,143.000000,143.000000,1.430000e+02,1.430000e+02
mean,5.926127e+05,1.715504e+06,34719.447552,365.118881,37.552448,24.475524,2.267385e+05,7.235235e+05,1.714731e+05,676.384615,1191.972028,1.489942e+06,2.404233e+06
std,1.036924e+06,3.694149e+06,45235.547286,1455.675655,74.148184,80.080666,7.552178e+05,1.571184e+06,1.660404e+05,1066.923179,2223.860300,2.386745e+06,4.422592e+06
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-2.604490e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,-4.409300e+04
25%,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,8.929250e+04,2.148255e+05
50%,2.500000e+05,6.014380e+05,17355.000000,16.000000,1.000000,0.000000,8.910000e+02,3.466630e+05,2.061210e+05,91.000000,266.000000,9.114530e+05,9.543540e+05
75%,8.000000e+05,1.636136e+06,52688.500000,50.500000,39.500000,12.500000,1.492040e+05,6.801640e+05,2.670975e+05,869.000000,1504.000000,1.858492e+06,2.217787e+06
max,8.000000e+06,3.076606e+07,228763.000000,14368.000000,528.000000,609.000000,7.427621e+06,1.384707e+07,1.060932e+06,5521.000000,15149.000000,1.725253e+07,3.076606e+07


In [28]:
print enron_data.keys()

['METTS MARK', 'BAXTER JOHN C', 'ELLIOTT STEVEN', 'CORDES WILLIAM R', 'HANNON KEVIN P', 'MORDAUNT KRISTINA M', 'MEYER ROCKFORD G', 'MCMAHON JEFFREY', 'HORTON STANLEY C', 'PIPER GREGORY F', 'HUMPHREY GENE E', 'UMANOFF ADAM S', 'BLACHMAN JEREMY M', 'SUNDE MARTIN', 'GIBBS DANA R', 'LOWRY CHARLES P', 'COLWELL WESLEY', 'MULLER MARK S', 'JACKSON CHARLENE R', 'WESTFAHL RICHARD K', 'WALTERS GARETH W', 'WALLS JR ROBERT H', 'KITCHEN LOUISE', 'CHAN RONNIE', 'BELFER ROBERT', 'SHANKMAN JEFFREY A', 'WODRASKA JOHN', 'BERGSIEKER RICHARD P', 'URQUHART JOHN A', 'BIBI PHILIPPE A', 'RIEKER PAULA H', 'WHALEY DAVID A', 'BECK SALLY W', 'HAUG DAVID L', 'ECHOLS JOHN B', 'MENDELSOHN JOHN', 'HICKERSON GARY J', 'CLINE KENNETH W', 'LEWIS RICHARD', 'HAYES ROBERT E', 'MCCARTY DANNY J', 'KOPPER MICHAEL J', 'LEFF DANIEL P', 'LAVORATO JOHN J', 'BERBERIAN DAVID', 'DETMERING TIMOTHY J', 'WAKEHAM JOHN', 'POWERS WILLIAM', 'GOLD JOSEPH', 'BANNANTINE JAMES M', 'DUNCAN JOHN H', 'SHAPIRO RICHARD S', 'SHERRIFF JOHN R', 'SHELBY 

In [31]:
# drop row for 'THE TRAVEL AGENCY IN THE PARK'

park_index = enron_data.keys().index('THE TRAVEL AGENCY IN THE PARK')
print park_index
df_imp_sub = df_imp.drop(df_imp.index[[park_index]])
df_imp_sub.describe()

99


,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
count,1.420000e+02,1.420000e+02,142.000000,142.000000,142.000000,142.000000,1.420000e+02,1.420000e+02,1.420000e+02,142.000000,142.000000,1.420000e+02,1.420000e+02
mean,5.967860e+05,1.727585e+06,34963.950704,367.690141,37.816901,24.647887,2.257853e+05,7.286187e+05,1.726806e+05,681.147887,1200.366197,1.497885e+06,2.421165e+06
std,1.039388e+06,3.704389e+06,45300.747867,1460.502581,74.342949,80.337515,7.578048e+05,1.575560e+06,1.659968e+05,1069.172948,2229.457770,2.393296e+06,4.433593e+06
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-2.604490e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,-4.409300e+04
25%,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.115000e+03,0.000000e+00,0.000000,0.000000,8.839225e+04,2.288695e+05
50%,2.750000e+05,6.046375e+05,18094.500000,16.500000,2.500000,0.000000,8.825000e+02,3.535955e+05,2.083105e+05,102.500000,289.000000,9.138250e+05,9.551135e+05
75%,8.000000e+05,1.636585e+06,52905.250000,51.250000,39.750000,12.750000,1.454285e+05,6.892030e+05,2.670998e+05,871.500000,1513.000000,1.863625e+06,2.218031e+06
max,8.000000e+06,3.076606e+07,228763.000000,14368.000000,528.000000,609.000000,7.427621e+06,1.384707e+07,1.060932e+06,5521.000000,15149.000000,1.725253e+07,3.076606e+07


In [34]:
print 'Columns:', list(df_imp_sub.columns.values)
print 'Shape:', df_imp_sub.shape
print 'Number of POI in the sub dataset:', (df_imp_sub['poi'] == 1).sum()
print 'Number of non-POI in the sub dataset:', (df_imp_sub['poi'] == 0).sum()

 Columns: ['bonus', 'exercised_stock_options', 'expenses', 'from_messages', 'from_poi_to_this_person', 'from_this_person_to_poi', 'other', 'poi', 'restricted_stock', 'salary', 'shared_receipt_with_poi', 'to_messages', 'total_payments', 'total_stock_value']
Shape: (142, 14)
Number of POI in the sub dataset: 16
Number of non-POI in the sub dataset: 126


### Task 3: Create new feature(s)

Create column 'poi_ratio' to store ratio in percentages. 

Also will be added next features:
the fraction of all emails to a person that were sent from a person of interest
the fraction of all emails that a person sent that were addressed to persons of interest

The hypothesis behind these features was that there might be stronger email connections between POIs than between POIs and non-POIs, and a scatterplot of these two features suggests that there might be some truth to that hypothesis.

Additionally i want to scale 'salary' to range [0,100].

In [43]:
def computeFraction( poi_messages, all_messages ):
    """ given a number messages to/from POI (numerator)
        and number of all messages to/from a person (denominator),
        return the fraction of messages to/from that person
        that are from/to a POI
   """
    fraction = 0.
    if poi_messages != 0 and all_messages != 0:
        fraction = float(poi_messages) / float(all_messages)

    return fraction * 100

In [44]:
for index, row in df_imp_sub.iterrows():
    row['poi_ratio'] = computeFraction(row['from_poi_to_this_person'] + row['from_this_person_to_poi'], row['from_messages'] + row['to_messages'])
    row['fraction_to_poi'] = computeFraction(row['from_this_person_to_poi'], row['from_messages'])
    row['fraction_from_poi'] = computeFraction(row['from_poi_to_this_person'], row['to_messages'])

df_imp_sub.describe()

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,poi_ratio,fraction_to_poi,fraction_from_poi
count,1.420000e+02,1.420000e+02,142.000000,142.000000,142.000000,142.000000,1.420000e+02,1.420000e+02,1.420000e+02,142.000000,142.000000,1.420000e+02,1.420000e+02,84.000000,84.000000,84.000000
mean,5.967860e+05,1.727585e+06,34963.950704,367.690141,37.816901,24.647887,2.257853e+05,7.286187e+05,1.726806e+05,681.147887,1200.366197,1.497885e+06,2.421165e+06,4.808472,17.983987,3.823534
std,1.039388e+06,3.704389e+06,45300.747867,1460.502581,74.342949,80.337515,7.578048e+05,1.575560e+06,1.659968e+05,1069.172948,2229.457770,2.393296e+06,4.433593e+06,4.694606,21.091430,4.098911
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-2.604490e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,-4.409300e+04,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.115000e+03,0.000000e+00,0.000000,0.000000,8.839225e+04,2.288695e+05,NaN,NaN,NaN
50%,2.750000e+05,6.046375e+05,18094.500000,16.500000,2.500000,0.000000,8.825000e+02,3.535955e+05,2.083105e+05,102.500000,289.000000,9.138250e+05,9.551135e+05,NaN,NaN,NaN
75%,8.000000e+05,1.636585e+06,52905.250000,51.250000,39.750000,12.750000,1.454285e+05,6.892030e+05,2.670998e+05,871.500000,1513.000000,1.863625e+06,2.218031e+06,NaN,NaN,NaN
max,8.000000e+06,3.076606e+07,228763.000000,14368.000000,528.000000,609.000000,7.427621e+06,1.384707e+07,1.060932e+06,5521.000000,15149.000000,1.725253e+07,3.076606e+07,22.435175,100.000000,21.734104


In [42]:
df_imp_sub.head()

,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,poi,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,poi_ratio,fraction_to_poi,fraction_from_poi
0,600000.0,0.0,94299.0,29.0,38.0,1.0,1740.0,False,585062.0,365788.0,702.0,807.0,1061827.0,585062.0,4.665072,3.448276,4.708798
1,1200000.0,6680544.0,11200.0,0.0,0.0,0.0,2660303.0,False,3942714.0,267102.0,0.0,0.0,5634343.0,10623258.0,NaN,NaN,NaN
2,350000.0,4890344.0,78552.0,0.0,0.0,0.0,12961.0,False,1788391.0,170941.0,0.0,0.0,211725.0,6678735.0,NaN,NaN,NaN
3,0.0,651850.0,0.0,12.0,10.0,0.0,0.0,False,386335.0,0.0,58.0,764.0,0.0,1038185.0,1.288660,0.000000,1.308901
4,1500000.0,5538001.0,34039.0,32.0,32.0,21.0,11350.0,True,853064.0,243293.0,1035.0,1045.0,288682.0,6391065.0,4.921077,65.625000,3.062201


In [37]:
df['price'] = df.apply(lambda row: valuation_formula(row['x'], row['y']), axis=1)

poi_ratio = (df_imp_sub['from_poi_to_this_person'] + df_imp_sub['from_this_person_to_poi']) / (df_imp_sub['from_messages'] + df_imp_sub['to_messages'])
fraction_to_poi = (df_imp_sub['from_this_person_to_poi']) / (df_imp_sub['from_messages'])
fraction_from_poi = (df_imp_sub['from_poi_to_this_person']) / (df_imp_sub['to_messages'])

df_imp_sub['poi_ratio'] = pd.Series(poi_ratio) * 100
df_imp_sub['fraction_to_poi'] = pd.Series(fraction_to_poi) * 100
df_imp_sub['fraction_from_poi'] = pd.Series(fraction_from_poi) * 100

scale = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 100), copy=True)
salary_scaled = scale.fit_transform(df_imp_sub['salary'])

df_imp_sub.describe()

/Users/LiWang/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/LiWang/anaconda/lib/python2.7/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/LiWang/anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,bonus,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,other,restricted_stock,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value,poi_ratio,fraction_to_poi,fraction_from_poi
count,1.420000e+02,1.420000e+02,142.000000,142.000000,142.000000,142.000000,1.420000e+02,1.420000e+02,1.420000e+02,142.000000,142.000000,1.420000e+02,1.420000e+02,84.000000,84.000000,84.000000
mean,5.967860e+05,1.727585e+06,34963.950704,367.690141,37.816901,24.647887,2.257853e+05,7.286187e+05,1.726806e+05,681.147887,1200.366197,1.497885e+06,2.421165e+06,4.808472,17.983987,3.823534
std,1.039388e+06,3.704389e+06,45300.747867,1460.502581,74.342949,80.337515,7.578048e+05,1.575560e+06,1.659968e+05,1069.172948,2229.457770,2.393296e+06,4.433593e+06,4.694606,21.091430,4.098911
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,-2.604490e+06,0.000000e+00,0.000000,0.000000,0.000000e+00,-4.409300e+04,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,8.115000e+03,0.000000e+00,0.000000,0.000000,8.839225e+04,2.288695e+05,NaN,NaN,NaN
50%,2.750000e+05,6.046375e+05,18094.500000,16.500000,2.500000,0.000000,8.825000e+02,3.535955e+05,2.083105e+05,102.500000,289.000000,9.138250e+05,9.551135e+05,NaN,NaN,NaN
75%,8.000000e+05,1.636585e+06,52905.250000,51.250000,39.750000,12.750000,1.454285e+05,6.892030e+05,2.670998e+05,871.500000,1513.000000,1.863625e+06,2.218031e+06,NaN,NaN,NaN
max,8.000000e+06,3.076606e+07,228763.000000,14368.000000,528.000000,609.000000,7.427621e+06,1.384707e+07,1.060932e+06,5521.000000,15149.000000,1.725253e+07,3.076606e+07,22.435175,100.000000,21.734104


In [ ]:
# 
plt.scatter('salary', 'fraction_to_poi', color='blue')
plt.scatter('salary', 'fraction_from_poi', color='red')
plt.xlabel('Salary')
plt.ylabel('Fraction')
plt.show()

### Task 4: Try a varity of classifiers

In [ ]:
### Task 4: Try a varity of classifiers
### Please name your classifier clf for easy export below.
### Note that if you want to do PCA or other multi-stage operations,
### you'll need to use Pipelines. For more info:
### http://scikit-learn.org/stable/modules/pipeline.html



In [ ]:
# Split the data at first on training and test data. We will use Stratified Shuffle Split due to small dataset
labels = df_im_sub['poi'].copy(deep=True).astype(int).as_matrix()
features = (df_im_sub.drop('poi', axis=1)).fillna(0).copy(deep=True).as_matrix()
shuffle = sklearn.cross_validation.StratifiedShuffleSplit(labels, 4, test_size=0.3, random_state=0)

print labels
print features

In [ ]:
# Data transformation with held out data
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(features_train)
features_train_transformed = scaler.transform(features_train)
clf = svm.SVC(C=1).fit(features_train_transformed, labels_train)
features_test_transformed = scaler.transform(features_test)
clf.score(features_test_transformed, labels_test)  

In [ ]:
# Try GaussianNB
from sklearn.naive_bayes import GaussianNB

gnb_clf = GaussianNB()
scores = sklearn.cross_validation.cross_val_score(gnb_clf, features, labels)
print scores

In [ ]:
# Try ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

erf_clf = ExtraTreesClassifier(n_estimators=10, max_depth=None, min_samples_split=1, random_state=0)
scores = sklearn.cross_validation.cross_val_score(erf_clf, features, labels)
print scores

In [ ]:
# Try RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=10)
scores = sklearn.cross_validation.cross_val_score(rf_clf, features, labels)

print scores

In [ ]:
# Try AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

ab_clf = AdaBoostClassifier(n_estimators=100)
scores = sklearn.cross_validation.cross_val_score(ab_clf, features, labels)
print scores 


Choosen Algorithms:
Random Forest
AdaBoost
Why not SVM? It's too slow for tuning, i want more computational time efficient algorithm.
SVM and NB seems not to be good choice from initial scores.


### Task 5: Tune your classifier to achieve better than .3 precision and recall 

In [ ]:
### Task 5: Tune your classifier to achieve better than .3 precision and recall 
### using our testing script. Check the tester.py script in the final project
### folder for details on the evaluation method, especially the test_classifier
### function. Because of the small size of the dataset, the script uses
### stratified shuffle split cross validation. For more info: 
### http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html

# Example starting point. Try investigating other evaluation techniques!
from sklearn.cross_validation import train_test_split
features_train, features_test, labels_train, labels_test = \
    train_test_split(features, labels, test_size=0.3, random_state=42)



#### Get the best fitting parameters and Testing for RandomForest

In [ ]:
# get the best parameters
from sklearn.model_selection import KFold

cv = KFold(n_splits=5)

parameters = {'n_estimators': [10,20,30,40,50],
               'min_samples_split' :[2,3,4,5],
               'min_samples_leaf' : [1,2,3]
             }

rfclf = RandomForestClassifier()
grid_obj = GridSearchCV(rfclf, parameters, cv=cv)
grid_fit = grid_obj.fit(features_train, labels_train)
best_rfclf = grid_fit.best_estimator_ 

best_rfclf.fit(features_train,labels_train)

In [ ]:
# RandomForestClassifier use the best parameter
from sklearn.cross_validation import KFold

rf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=40, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

rf.fit(features_train, labels_train)

kf = KFold(titanic.shape[0], n_folds=3, random_state=1)
scores = cross_validation.cross_val_score(rf, features, labels_train, cv=kf)
# Take the mean of the scores (because we have one for each fold)
print(scores.mean())

output = best_rfclf.predict(features_test)
print len(output)

In [ ]:
from sklearn import grid_search

cv = sklearn.cross_validation.StratifiedShuffleSplit(labels, n_iter=10)

def scoring(estimator, features_test, labels_test):
     labels_pred = estimator.predict(features_test)
     p = sklearn.metrics.precision_score(labels_test, labels_pred, average='micro')
     r = sklearn.metrics.recall_score(labels_test, labels_pred, average='micro')
     if p > 0.3 and r > 0.3:
            return sklearn.metrics.f1_score(labels_test, labels_pred, average='macro')
     return 0

'''
parameters = {'max_depth': [2,3,4,5,6],'min_samples_split':[2,3,4,5], 'n_estimators':[10,20,50], 'min_samples_leaf':[1,2,3,4], 'criterion':('gini', 'entropy')}
rf_clf = RandomForestClassifier()
rfclf = grid_search.GridSearchCV(rf_clf, parameters, scoring = scoring, cv = cv)
rfclf.fit(features, labels)

print rfclf.best_estimator_
print rfclf.best_score_
'''

#### Get the best fitting parameters and Testing for Adaboost

In [ ]:

from sklearn import grid_search
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
parameters = {'n_estimators' : [5, 10, 30, 40, 50, 100,150], 'learning_rate' : [0.1, 0.5, 1, 1.5, 2, 2.5], 'algorithm' : ('SAMME', 'SAMME.R')}
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=8))
adaclf = grid_search.GridSearchCV(ada_clf, parameters, scoring = scoring, cv = cv)
adaclf.fit(features, labels)

print adaclf.best_estimator_
print adaclf.best_score_

In [ ]:
#prepare parameter for dump classifier
clf_dump = adaclf

features_list = list(df_imp_sub.columns.values)

list_cols = list(df_subset.columns.values)
features_list.remove('poi')
features_list.insert(0, 'poi')
data = df_subset[features_list].fillna(0).to_dict(orient='records')
enron_data_sub = {}
counter = 0
for item in data:
    enron_data_sub[counter] = item
    counter += 1
    
my_dataset = enron_data_sub    

### Task 6: Dump your classifier, dataset, and features_list for checking

In [ ]:
### Task 6: Dump your classifier, dataset, and features_list so anyone can
### check your results. You do not need to change anything below, but make sure
### that the version of poi_id.py that you submit can be run on its own and
### generates the necessary .pkl files for validating your results.

dump_classifier_and_data(clf_dump, my_dataset, features_list)

### List of Resources


sklearn documentation: http://scikit-learn.org/stable/index.html
pandas documentation: http://pandas.pydata.org
Jason Brownlee on Machine Learning Process:  How to Identify Outliers in your Data http://machinelearningmastery.com/how-to-identify-outliers-in-your-data/
